In [1]:
import pandas as pd
import numpy as np
import csv
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import logging
import math
from lmfit import minimize, fit_report, Parameters
from aim2_population_model_spatial_aff_parallel import get_mod_spike
from model_constants import (MC_GROUPS, LifConstants)
from popul_model import pop_model
from aim2_population_model_spatial_aff_parallel import Afferent, SimulationConfig, Simulation

In [7]:
#Global Variables
lmpars_init_dict = {}
lmpars = Parameters()
lmpars.add('tau1', value=8, vary=False)
lmpars.add('tau2', value=200, vary=False)
lmpars.add('tau3', value=1744.6, vary=False)
lmpars.add('tau4', value=np.inf, vary=False)
lmpars.add('k1', value=.74, vary=False, min=0) #a constant
lmpars.add('k2', value=2.75, vary=False, min=0) #b constant
lmpars.add('k3', value=.07, vary=False, min=0) #c constant
lmpars.add('k4', value=.0312, vary=False, min=0)
lmpars_init_dict['t3f12v3final'] = lmpars


In [ ]:
def radial_stress_vf_model(afferent_type, vf_tip_size, scaling_factor = 0.3):
    """ Read in the Radial which has sample stress traces for every 2mm from a center point
    to calculate firing"""

    #regex pattern for exstracting the distance from the middle point
    distance_regex = r'\d\.\d{2}'

    #Reading in the radial stress file
    radial_stress = pd.read_csv(f"data/vfspatial/{vf_tip_size}_radial_stress.csv")
    radial_time = radial_stress['Time (ms)'].to_numpy()
    
    stress_data = {}
    iff_data = {}
    not_generated_radii = []

    # Inner loop to iterate through radial distances
    for col in radial_stress.columns[1:]:
        distance_from_center = float(re.findall(distance_regex, col)[0])

        # Initialize lists for each coordinate-distance pair
        afferent_type = []
        spikes = []
        mean_firing_frequency = []
        peak_firing_frequency = []
        first_spike_time = []
        last_spike_time = []

        # if radial_spatial_flag:
        #     radial_stress_vals = radial_stress[col]
        #     radial_stress_max = np.max(radial_stress_vals)
        #     distance_scaling_factor = spatial_stress_max / radial_stress_max
        #     radial_spatial_flag = False
        
        #where the scaling happens
        scaled_stress = radial_stress[col] *  scaling_factor

        stress_data[distance_from_center] = {
            "Time": radial_time,
            distance_from_center: scaled_stress.to_numpy()
        }

        lmpars = lmpars_init_dict['t3f12v3final']
        if afferent_type == "RA":
            lmpars['tau1'].value = 2.5
            lmpars['tau2'].value = 200
            lmpars['tau3'].value = 1
            lmpars['k1'].value = 35
            lmpars['k2'].value = 0
            lmpars['k3'].value = 0.0
            lmpars['k4'].value = 0

        groups = MC_GROUPS
        mod_spike_time, mod_fr_inst = get_mod_spike(lmpars, groups, stress_data[distance_from_center]["Time"], stress_data[distance_from_center][distance_from_center])

        if len(mod_spike_time) == 0 or len(mod_fr_inst) == 0:
            logging.warning(f"SPIKES COULD NOT BE GENERATED FOR {vf_tip_size}")
            not_generated_radii.append(distance_from_center)
            continue

        if len(mod_spike_time) != len(mod_fr_inst):
            if len(mod_fr_inst) > 1:
                mod_fr_inst_interp = np.interp(mod_spike_time, radial_time, mod_fr_inst)
            else:
                mod_fr_inst_interp = np.zeros_like(mod_spike_time)
        else:
            mod_fr_inst_interp = mod_fr_inst

        features, _ = pop_model(mod_spike_time, mod_fr_inst_interp)

        # Append single values to the lists
        afferent_type.append(afferent_type)
        spikes.append(len(mod_spike_time) if len(mod_spike_time) != 0 else None)
        mean_firing_frequency.append(features["Average Firing Rate"])
        peak_firing_frequency.append(np.max(mod_fr_inst_interp))
        first_spike_time.append(mod_spike_time[0] if len(mod_spike_time) != 0 else None)
        last_spike_time.append(mod_spike_time[-1])

        # Store each coordinate-distance dictionary within iff_data
        iff_data[distance_from_center] = {
            'afferent_type': afferent_type,
            'num_of_spikes': spikes[0],
            'mean_firing_frequency': mean_firing_frequency[0],
            'peak_firing_frequency': peak_firing_frequency[0],
            'first_spike_time': first_spike_time[0],
            'last_spike_time': last_spike_time[0]
        }

    return stress_data, iff_data, not_generated_radii

In [ ]:

stresses, iffs = radial_stress_vf_model("SA",4.17, scaling_factor = .3)

In [ ]:
scaling_factors = np.linspace(0,.5,1000)

for sf in scaling_factors:
    


{0.0: {'afferent_type': [[...]], 'num_of_spikes': 177, 'mean_firing_frequency': 182.90411035143228, 'peak_firing_frequency': 0.3333333333333333, 'first_spike_time': 82.0, 'last_spike_time': 2491.5}, 0.33: {'afferent_type': [[...]], 'num_of_spikes': 110, 'mean_firing_frequency': 118.22937459445127, 'peak_firing_frequency': 0.2, 'first_spike_time': 93.5, 'last_spike_time': 2490.0}, 0.51: {'afferent_type': [[...]], 'num_of_spikes': 87, 'mean_firing_frequency': 97.31234183091911, 'peak_firing_frequency': 0.16666666666666666, 'first_spike_time': 99.5, 'last_spike_time': 2487.5}, 0.71: {'afferent_type': [[...]], 'num_of_spikes': 73, 'mean_firing_frequency': 84.40053297034072, 'peak_firing_frequency': 0.14285714285714285, 'first_spike_time': 104.0, 'last_spike_time': 2482.0}, 0.9: {'afferent_type': [[...]], 'num_of_spikes': 57, 'mean_firing_frequency': 70.65870543893888, 'peak_firing_frequency': 0.11764705882352941, 'first_spike_time': 110.5, 'last_spike_time': 2485.0}, 1.09: {'afferent_type'